In [1]:
import sys
sys.path.append('../../')

In [2]:
#from scripts.utils import data_path#, set_mpl
from scripts.data_and_models import read_london, predict_london

import plotly
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import display for pandas dataframes
from IPython.display import display

#set_mpl()


../..
matplotlib settings set
matplotlib settings set


In [3]:

london_dict = read_london(num_homes = 300)

horizon = 24*7
start_date = '2013-03-07'
predict_dict = predict_london(london_dict, timestamp = start_date,  horizon = horizon)



2023-03-09 09:30:21.407 
  command:

    streamlit run /Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]



    Loading London data from ../../0_data/.
    Weather from `meteostat` package.

    STD and ToU tariffs are separated.
    Data resampled (mean) to 1H resolution from original 30min resolution.

    reutrns:
    df_std: pd.DataFrame with STD tariff data
    df_tou: pd.DataFrame with ToU tariff data
    df_weather: pd.DataFrame with weather data
    df_twitter: pd.DataFrame with twitter data (see `0_data/2.2_london_twitter.ipynb` for details)
    
    


In [4]:


train_df = predict_dict['train'].pd_dataframe()
test_df = predict_dict['test'].pd_dataframe()


fig = go.Figure()
fig.update_layout(
autosize=False,
width=1000,
height=500)

fig.add_trace(go.Scatter(x=train_df.tail(24).index, y=train_df.tail(24)['power_avg'],
                mode='lines',
                name='train'))
fig.add_trace(go.Scatter(x=test_df.head(horizon).index, y=test_df.head(horizon)['power_avg'],
                mode='lines',
                name='test'))   

list_of_models = ['Naive', 'EAI (no human behaviour)', 'EAI',  'Ensemble']

for model_name in list_of_models:
    model_df = predict_dict[model_name]['pred_test'].pd_dataframe()
    fig.add_trace(go.Scatter(x=model_df.index, y=model_df['power_avg'],
                    mode='lines',
                    name=model_name))

fig.update_layout(
title="Train and test data",
xaxis_title="Date",
yaxis_title="Power consumption",
legend_title="Legend Title",
font=dict(
    family="Courier New, monospace",
    size=18,
    color="RebeccaPurple"
)
)

fig.show()

metrics = predict_dict['metrics']

display(metrics.loc['mape', :][['Week ahead', 'Last week']])
display(metrics.loc['smape', :][['Week ahead', 'Last week']])


tweets_df = london_dict['darts_dict']['twitter_covariate'].pd_dataframe()
twets_clipped = tweets_df.loc[pd.Timestamp(start_date):pd.Timestamp(start_date)+pd.Timedelta(hours = horizon)]

fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(go.Scatter(x=twets_clipped.index, y=twets_clipped['tweets_total'],
                    mode='lines',
                    name='tweets'),
                    secondary_y=False)

fig.add_trace(go.Scatter(x=test_df.head(horizon).index, y=test_df.head(horizon)['power_avg'],
                mode='lines',
                name = 'Energy'), secondary_y=True)   

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Tweets per hour", secondary_y=False)
fig.update_yaxes(title_text="Hourly energy consumption", secondary_y=True)







,Week ahead,Last week
model,,
EAI,8.18,3.61
EAI (no human behaviour),10.83,4.22
Ensemble,7.84,5.55
Naive,8.63,8.96


,Week ahead,Last week
model,,
EAI,7.75,3.59
EAI (no human behaviour),9.95,4.10
Ensemble,7.54,5.77
Naive,8.40,9.59
